In [1]:
!pip install catboost
!pip install scikit-learn
!pip install nltk
!pip install pymorphy2
!python -m nltk.downloader stopwords

In [2]:
import pandas as pd
import numpy as np
import pymorphy2
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

# **Data prepararion**

**Read data from files**

In [5]:
pd_train=pd.read_csv('train.csv')
pd_train

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...
38736,669736851,autosport,Ира Сидоркова объясняет как сказалась на ее ма...
38737,558919241,tennis,24 я ракетка мира хорват Марин Чилич обыграл и...
38738,776944963,volleyball,Стал известен календарь мужской сборной России...


In [6]:
pd_test=pd.read_csv('test.csv')
pd_test

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


**Group data by same "oid" values**
ₙCategory" values is same for same "oid" and i took min(), and "text" are merged

In [7]:
gr=pd_train.groupby(by=['oid']).agg({'category':'min','text':' '.join})
pd_train_gr=pd.DataFrame(gr).reset_index()
pd_train_gr

,oid,category,text
0,16594,extreme,Пост для обсуждения наболевшего по китайским ф...
1,66573,winter_sport,Паралимпийский комитет России намерен сделать ...
2,84576,autosport,В FIA утвердили регламент на моторы Всемирный ...
3,203396,extreme,От чего у мужчины зависит размер кулаков? ? ? ...
4,621640,athletics,Всем бег друзья 33 Marat Satokenoid делисьбего...
...,...,...,...
3869,998385654,volleyball,Знаете ли вы историю возникновения пляжного во...
3870,999156041,boardgames,Поделимся с вами хорошо написанными и оформлен...
3871,999377741,volleyball,День не задался Пошла играть в волейбол и влет...
3872,999704370,hockey,Воспитанники ДЮСШ Лада возложили цветы к Вечно...


In [8]:
gr=pd_test.groupby(by=['oid'],group_keys=False)['text'].agg(' '.join)
pd_test_gr=pd.DataFrame(gr.to_frame().reset_index())
pd_test_gr

,oid,text
0,1622114,Нет ничего более стимулирующего чем когда все ...
1,1663817,Радиопереговоры Пьера Гасли в момент аварии с ...
2,3174332,Сет Карри вылавливает сложный мяч и забивает и...
3,3469228,Всем привет 33 Взяла 44. 000 рублей на 6 месяц...
4,3905302,Российский гроссмейстер Сергей Карякин заявил ...
...,...,...
2621,998309713,Типичный день в ТРАЙХАРД Ждешь новые серии? me...
2622,998565619,Как этот игрок залез на товарища если тот даже...
2623,999112505,Всего пара недель осталась до завершения кампа...
2624,999361308,Дарья Касаткина в соцсетях ﻿отреагировала на в...


**Clean punctuation and stopwords. Change to lower case and lemmatize words in text.**

In [9]:
russian_stopwords = stopwords.words("russian")
lemma = pymorphy2.MorphAnalyzer()

def preprocess_text(doc):
    trans_table = str.maketrans({}.fromkeys(string.punctuation))
    text=doc.translate(trans_table).lower()    
    list_text = [lemma.parse(i)[0].normal_form for i in text.split() if (i not in russian_stopwords) and len(i)>1]
    text = " ".join([i.strip() for i in list_text if (i not in russian_stopwords) and len(i)>1 ])
    return text

In [10]:
pd_train_res=pd_train_gr.copy()
pd_train_res.text=pd_train_res.text.apply(preprocess_text)
pd_train_res

,oid,category,text
0,16594,extreme,пост обсуждение наболевший китайский фандом не...
1,66573,winter_sport,паралимпийский комитет россия намеренный сдела...
2,84576,autosport,fia утвердить регламент мотор всемирный совет ...
3,203396,extreme,мужчина зависеть размер кулак кулак издревле с...
4,621640,athletics,весь бег друг 33 marat satokenoid делисьбегом ...
...,...,...,...
3869,998385654,volleyball,знать история возникновение пляжный волейбол п...
3870,999156041,boardgames,поделиться написать оформить отчёт игра которы...
3871,999377741,volleyball,день задаться пойти играть волейбол влететь че...
3872,999704370,hockey,воспитанник дюсш лада возложить цветок вечный ...


In [11]:
pd_test_res=pd_test_gr.copy()
pd_test_res.text=pd_test_res.text.apply(preprocess_text)
pd_test_res

,oid,text
0,1622114,стимулировать идти жить лёгкий атлетика пусть ...
1,1663817,радиопереговоры пьер гаснуть момент авария рек...
2,3174332,сет карри вылавливать сложный мяч забивать дуг...
3,3469228,весь привет 33 взять 44 000 рубль месяц 33 оче...
4,3905302,российский гроссмейстер сергей карякин заявить...
...,...,...
2621,998309713,типичный день трайхард ждать новый серия memes...
2622,998565619,игрок залезть товарищ присесть xyp9x делать вы...
2623,999112505,пара неделя остаться завершение кампания игра ...
2624,999361308,дарья касаткин соцсеть ﻿отреагировать выход по...


# **Dictanories**

**Dictanories**

In [12]:
import collections
import nltk
nltk.download('punkt')

names=['athletics','autosport','basketball','boardgames','esport','extreme','football','hockey','martial_arts','motosport','tennis','volleyball','winter_sport']

#calculate dictanories of words for each category
vals=dict.fromkeys(names,collections.Counter())
temp=dict.fromkeys(names,collections.Counter())
for nn in names:
  pd_cat=pd_train_res[pd_train_res["category"].str.contains(nn)].copy()
  pd_counts = collections.Counter()
  for sent in pd_cat["text"]:
    words = nltk.word_tokenize(sent)  
    pd_counts.update(words)  
  vals[nn]=pd_counts.copy()
  temp[nn]=pd_counts.copy()

#calculate dufferences between dictanories of words for each by each category
for i in names:  
  for j in names:
     if i!=j: vals[i].subtract(temp[j])
train_dict_set=dict.fromkeys(names,())

# Take only wors with possitive counts for each category
for i in names:
  train_dict_set[i]=set(x[0] for x in vals[i].most_common() if x[1]>0)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
#check lenghts of sets train_dict_set[x]
print([len(train_dict_set[x]) for x in names])

[5436, 4988, 3089, 5100, 3316, 5939, 4986, 4057, 7077, 4421, 2465, 2846, 4723]


# **Solve without ML**

In [37]:
def set_category(sent):
  words = nltk.word_tokenize(sent)  
  tres_cat=dict.fromkeys(names,[])
  for i in names:
    tres_cat[i]=list(train_dict_set[i]&set(words))
  maxx=0
  cat_name=""  
  for i in names:    
    if maxx<len(tres_cat[i]): 
      maxx=len(tres_cat[i])      
      cat_name=i
  if maxx==0: 
    print("Warning: Empty category")
    
  return cat_name

In [38]:
res_test=pd_test_res.copy()
res_test['category']=res_test['text'].apply(set_category)
res_test

,oid,text,category
0,1622114,стимулировать идти жить лёгкий атлетика пусть ...,athletics
1,1663817,радиопереговоры пьер гаснуть момент авария рек...,autosport
2,3174332,сет карри вылавливать сложный мяч забивать дуг...,basketball
3,3469228,весь привет 33 взять 44 000 рубль месяц 33 оче...,extreme
4,3905302,российский гроссмейстер сергей карякин заявить...,boardgames
...,...,...,...
2621,998309713,типичный день трайхард ждать новый серия memes...,esport
2622,998565619,игрок залезть товарищ присесть xyp9x делать вы...,esport
2623,999112505,пара неделя остаться завершение кампания игра ...,boardgames
2624,999361308,дарья касаткин соцсеть ﻿отреагировать выход по...,tennis


result is about 90-92%. 

# **CatBoostClassifier**

take union of intersections dyctanory for each category and text

In [18]:
def get_words(sent):
  words = nltk.word_tokenize(sent)  
  tres=[]
  for i in names:
    x=train_dict_set[i]&set(words)
    tres+=list(x)
  if len(tres)==0: 
    print("Warning: empty text")
    return ""
  return " ".join(tres)


In [19]:
new_test=pd_test_res.copy()
new_test['text2']=new_test['text'].apply(get_words)

In [20]:
new_train=pd_train_res.copy()
new_train['text2']=new_train['text'].apply(get_words)

In [ ]:
new_train



In [21]:
new_train.set_index('oid',inplace=True)
new_test.set_index('oid',inplace=True)

In [22]:
new_train.drop(columns=['text'], inplace=True)
new_test.drop(columns=['text'], inplace=True)

In [ ]:
new_train.set_axis(['category','text'],inplace=True,axis=1)

In [ ]:
new_test.set_axis(['text'],inplace=True,axis=1)

**Create pool**

In [23]:
from sklearn.model_selection import train_test_split
names=['athletics','autosport','basketball','boardgames','esport','extreme','football','hockey','martial_arts','motosport','tennis','volleyball','winter_sport']
x_col = ['text2']
y_col = ['category']
text_features = ['text2']
x_train, x_val, y_train, y_val = train_test_split(new_train[x_col], new_train[y_col], train_size=0.8)

In [24]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

train_pool = Pool(x_train, y_train, text_features=text_features)
validate_pool = Pool(x_val,y_val, text_features=text_features)
all_pool=Pool(new_train[x_col],new_train[y_col], text_features=text_features)

In [25]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=1000,
                           text_features=text_features,
                           task_type="GPU",
                           loss_function='MultiClass',                         
                           verbose=500,                            
                           random_seed=12,
                           score_function='L2',
                           )

In [26]:
model.fit(train_pool, eval_set=validate_pool)


Learning rate set to 0.117881
0:	learn: 2.0190548	test: 2.0184674	best: 2.0184674 (0)	total: 28.8ms	remaining: 28.8s
500:	learn: 0.0295724	test: 0.0953918	best: 0.0949845 (439)	total: 8.74s	remaining: 8.7s
999:	learn: 0.0118566	test: 0.0999668	best: 0.0949845 (439)	total: 17.2s	remaining: 0us
bestTest = 0.09498451479
bestIteration = 439
Shrink model to first 440 iterations.


In [ ]:
model.fit(all_pool)

0:	learn: 2.1526710	total: 38.2ms	remaining: 34.4s
500:	learn: 0.0358078	total: 8.33s	remaining: 6.64s
899:	learn: 0.0173985	total: 14.7s	remaining: 0us


In [27]:
model.score(x_train,y_train)

0.9983865763149403

In [28]:
model.score(x_val,y_val)

0.9754838709677419

In [29]:
from sklearn. metrics import classification_report
print(classification_report(y_val,model.predict(x_val)))

              precision    recall  f1-score   support

   athletics       0.98      0.95      0.97        64
   autosport       1.00      1.00      1.00        74
  basketball       0.98      0.93      0.95        54
  boardgames       0.98      1.00      0.99        60
      esport       0.93      0.98      0.96        57
     extreme       0.90      0.96      0.93        56
    football       0.97      0.95      0.96        62
      hockey       1.00      0.98      0.99        50
martial_arts       0.93      0.93      0.93        61
   motosport       1.00      0.98      0.99        59
      tennis       1.00      1.00      1.00        54
  volleyball       1.00      1.00      1.00        64
winter_sport       1.00      1.00      1.00        60

    accuracy                           0.98       775
   macro avg       0.98      0.97      0.98       775
weighted avg       0.98      0.98      0.98       775



**result is about 95% ** i stopped here. 